## Случай Лагранжа (Волчок Лагранжа)

In [18]:
%load_ext autoreload
%autoreload 2

import sympy as sp
from sympy import Derivative, Symbol, IndexedBase, Idx, Sum, Function, Matrix, Add, Eq
from sympy import diff, symbols, solve, linsolve, simplify, poly, pprint, factor, integrate, lambdify
from sympy import cos, sin, pi

# syspath trick
import sys
sys.path.append('../')

from tatarinov.utils.jupyter import display_list, display_obj
from tatarinov.core.variables import *
from tatarinov.problems.Lagrange_top.variables import *
from tatarinov.core.tatarinov import TatarinovSystem

from tatarinov.database.singleton import db
db.set_root('../data/Lagrange_top')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
Lval = Eq(L, A*(Derivative(theta,t)**2 + Derivative(psi,t)**2*sin(theta)**2)/2 +
          C*(Derivative(phi, t) + Derivative(psi, t)*cos(theta))**2/2 -
          m*g*cos(theta))
Lval

Eq(L, A*(sin(theta(t))**2*Derivative(psi(t), t)**2 + Derivative(theta(t), t)**2)/2 + C*(cos(theta(t))*Derivative(psi(t), t) + Derivative(phi(t), t))**2/2 - g*m*cos(theta(t)))

In [24]:
db.save(Lval, 'Lval', description='Лагранжиан')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/Lagrange_top


####  Создадим объект для механической системы с уравнениями Татаринова 

In [3]:
TS = TatarinovSystem()

In [4]:
TS.set_p([p[1], p[2], p[3]])
TS.set_q([phi, psi, theta])

In [5]:
constraints = []
constraints += [Eq(nu1, Derivative(psi,t)*sin(theta)*sin(phi) + Derivative(theta,t)*cos(phi))]
constraints += [Eq(nu2, Derivative(psi,t)*sin(theta)*cos(phi) - Derivative(theta,t)*sin(phi))]
constraints += [Eq(nu3, Derivative(phi,t) + Derivative(psi,t)*cos(theta))]
display_list(constraints)

Eq(nu1(t), sin(phi(t))*sin(theta(t))*Derivative(psi(t), t) + cos(phi(t))*Derivative(theta(t), t))

Eq(nu2(t), -sin(phi(t))*Derivative(theta(t), t) + sin(theta(t))*cos(phi(t))*Derivative(psi(t), t))

Eq(nu3(t), cos(theta(t))*Derivative(psi(t), t) + Derivative(phi(t), t))

In [22]:
db.save(constraints, 'constraints', description='Связи неразрешенные относительно старших производных')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/Lagrange_top


In [6]:
TS.set_constraints(constraints)
display_list(TS.constraints)

_dubug output:_ 

__Solving by: diff([phi(t), psi(t), theta(t)])__

---

Eq(Derivative(phi(t), t), -nu1(t)*sin(phi(t))/tan(theta(t)) - nu2(t)*cos(phi(t))/tan(theta(t)) + nu3(t))

Eq(Derivative(psi(t), t), (nu1(t)*sin(phi(t)) + nu2(t)*cos(phi(t)))/sin(theta(t)))

Eq(Derivative(theta(t), t), nu1(t)*cos(phi(t)) - nu2(t)*sin(phi(t)))

In [23]:
db.save(TS.constraints, 'solved_constraints', description='Связи разрешенные относительно старших производных')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/Lagrange_top


In [7]:
%%time
TS.set_L(simplify(Lval))

CPU times: user 1.74 s, sys: 3.98 ms, total: 1.74 s
Wall time: 1.74 s


In [8]:
%%time
L_star = TS.set_L_star(TS.sub_constraints(Lval))
display(TS.L_star)

Eq(L, A*nu1(t)**2/2 + A*nu2(t)**2/2 + C*nu3(t)**2/2 - g*m*cos(theta(t)))

CPU times: user 1.87 s, sys: 0 ns, total: 1.87 s
Wall time: 1.88 s


In [9]:
TS.set_omega_equations(omegas=[omega[1], omega[2], omega[3]],
                      equations=[nu1, nu2, nu3])
# display_list(TS.omega_equations)

In [10]:
TS.set_v_equations(vs=[v[1], v[2], v[3]],
                  equations=[el.diff() for el in TS.q])
# display_list(TS.v_equations)

In [11]:
F = Matrix([0,0])
TS.set_F(F)
# TS.F

In [12]:
TS.create_P()
display_list(TS.P)

_dubug output:_ 

Eq(Sum(P[k]*omega[k], (k, 1, 3)), Sum(p[k]*v[k], (k, 1, 3)))

---

_dubug output:_ 

Eq(nu1(t)*P[1] + nu2(t)*P[2] + nu3(t)*P[3], (nu1(t)*sin(phi(t)) + nu2(t)*cos(phi(t)))*p[2]/sin(theta(t)) + (nu1(t)*cos(phi(t)) - nu2(t)*sin(phi(t)))*p[3] + (-nu1(t)*sin(phi(t))/tan(theta(t)) - nu2(t)*cos(phi(t))/tan(theta(t)) + nu3(t))*p[1])

---

Eq(P[1], (-sin(phi(t))*sin(theta(t))*p[1] + sin(phi(t))*tan(theta(t))*p[2] + sin(theta(t))*cos(phi(t))*tan(theta(t))*p[3])/(sin(theta(t))*tan(theta(t))))

Eq(P[2], -(sin(phi(t))*sin(theta(t))*tan(theta(t))*p[3] + sin(theta(t))*cos(phi(t))*p[1] - cos(phi(t))*tan(theta(t))*p[2])/(sin(theta(t))*tan(theta(t))))

Eq(P[3], p[1])

In [15]:
TS.create_Q(r['p'])
display_list(TS.Q)

0

0

0

In [16]:
for i in range(3):
    TS.tatarinov_equation(i)

_dubug output:_ 

__Уравнение #0__

---

_dubug output:_ 

__Левая часть уравнения__

...

---

_dubug output:_ 

__Правая часть уравнения__

(-sin(phi(t))*sin(theta(t))*p[1] + sin(phi(t))*tan(theta(t))*p[2] + sin(theta(t))*cos(phi(t))*tan(theta(t))*p[3])*nu2(t)/(sin(theta(t))*tan(theta(t))) - (-sin(phi(t))*sin(theta(t))*tan(theta(t))*p[3] - sin(theta(t))*cos(phi(t))*p[1] + cos(phi(t))*tan(theta(t))*p[2])*nu1(t)/(sin(theta(t))*tan(theta(t)))

---

In [17]:
TS.display_equations()

Eq((A - C)*nu2(t)*nu3(t), A*Derivative(nu1(t), t) - g*m*sin(theta(t))*cos(phi(t)))

Eq((A - C)*nu1(t)*nu3(t), -A*Derivative(nu2(t), t) - g*m*sin(phi(t))*sin(theta(t)))

Eq(C*Derivative(nu3(t), t), 0)

In [25]:
db.save(Matrix(TS.tatarinov_equations), 'Teqs', description='Уравнения Татаринова для волчка Лагранжа')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/Lagrange_top
